In [1]:
include("../src/UNSflow.jl")
using UNSflow


In [2]:
cref = 1.
bref = 10.
sref = 10.

patch1 = patch(0., -5., 0., 0., "FlatPlate", 1., 0., 0.11, 10)
patch2 = patch(0., 0., 0., 0., "FlatPlate", 1., 0., 0.11, 10)
patch3 = patch(0., 5., 0., 0., "FlatPlate", 1., 0., 0.11, 5)
patchdata = [patch1; patch2; patch3]

alphadef = ConstDef(15.*pi/180)
hdef = ConstDef(0.)
udef = ConstDef(1.)
kin = KinemDef3D(alphadef, hdef, udef)







UNSflow.KinemDef3D(UNSflow.ConstDef(0.2617993877991494),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0),"Constant",0,[0.0])

In [3]:
surf = ThreeDSurf(cref, bref, sref, patchdata, kin, 1., 70, 35, 21)
field = ThreeDFlowField()
dtstar = 0.0075
nsteps =round(Int,2.5/dtstar) + 1

334

In [4]:
mat = Array(Float64, 0, 4)

    mat = mat'
    
    surf2d = TwoDSurf[]
    field2d = TwoDFlowField[]
    kinem2d = KinemDef[]

    dt = dtstar*surf.cref/surf.uref

    t = 0.

    AR = surf.bref/surf.cref

    bc = zeros(surf.nspan)
    a03d = zeros(surf.nspan)
    cl = zeros(surf.nspan)
    cd = zeros(surf.nspan)
    cm = zeros(surf.nspan)

    lhs = zeros(surf.nspan, surf.nbterm)
    rhs = zeros(surf.nspan)
    bcoeff = zeros(surf.nbterm)
    
    if surf.kindef.vartype == "Constant"

        for i = 1:surf.nspan
            # Kinematics at all strips is the same
            
            push!(kinem2d, KinemDef(surf.kindef.alpha, surf.kindef.h, surf.kindef.u))
            push!(surf2d, TwoDSurf(surf.patchdata[1].coord_file, surf.patchdata[1].pvt, kinem2d[i], [surf.patchdata[1].lc;]))
            #If 3D flow field is defined with disturbances or external vortices, these should be transferred to the 2D flowfield
            push!(field2d, TwoDFlowField())
        end
    end


In [5]:
 t = t + dt

        for i = 1:surf.nspan
            #Update kinematic parameters
            update_kinem(surf2d[i], t)
        
            #Update flow field parameters if any
            update_externalvel(field2d[i], t)

            #Update bound vortex positions
            update_boundpos(surf2d[i], dt)

            #Add a TEV with dummy strength
            place_tev(surf2d[i], field2d[i], dt)
        end
        
        kelv = KelvinConditionLLTldvm(surf, surf2d, field2d)
        
        #Solve for TEV strength to satisfy Kelvin condition
        
        soln = nlsolve(not_in_place(kelv), -0.01*ones(surf.nspan))
        
        for i = 1:surf.nspan
            field2d[i].tev[length(field2d[i].tev)].s = soln.zero[i]

            #Update incduced velocities on airfoil
            update_indbound(surf2d[i], field2d[i])
            
            #Calculate downwash
            update_downwash(surf2d[i], [field2d[i].u[1],field2d[i].w[1]])
            
            #Calculate first two fourier coefficients
            update_a0anda1(surf2d[i])

            bc[i] = surf2d[i].a0[1] + 0.5*surf2d[i].aterm[1]
        end


 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in slice(::Array{Float64,2}, ::Vararg{Any,N}) at ./deprecated.jl:30
 in sumabs2j at /Users/kramesh/.julia/v0.5/NLsolve/src/utils.jl:1 [inlined]
 in trust_region_(::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Float64, ::Bool) at /Users/kramesh/.julia/v0.5/NLsolve/src/trust_region.jl:105
 in #nlsolve#17(::Symbol, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Function, ::Float64, ::Bool, ::NLsolve.#nlsolve, ::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}) at /Users/kramesh/.julia/v0.5/NLsolve/src/nlsolve_func_defs.jl:24
 in nlsolve(::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}) at /Users/kramesh/.julia/v0.5/NLsolve/src/nlsolve_func_defs.jl:13
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kramesh/.julia/v0.5/IJulia/src/execute_request.jl:169
 i

In [6]:
  for i = 1:surf.nspan
            for n = 1:surf.nbterm
                lhs[i,n] = sin(n*surf.psi[i])*(sin(surf.psi[i]) + (n*pi/(2*AR)))
            end
            rhs[i] = pi*sin(surf.psi[i])*bc[i]/(2*AR)
        end
        bcoeff[:] = \(lhs, rhs)   

        for i = 1:surf.nspan
            a03d[i] = 0
            for n = 1:surf.nbterm
            a03d[i] = a03d[i] - real(n)*bcoeff[n]*sin(n*surf.psi[i])/sin(surf.psi[i])
            end
        end

In [7]:
nshed = Int(0) 
        for i = 1:surf.nspan
            #Update 3D effect on A0
            surf2d[i].a0[1] = surf2d[i].a0[1] + a03d[i]

            #2D iteration if LESP_crit is exceeded
            if abs(surf2d[i].a0[1]) > surf2d[i].lespcrit[1]
                #Remove the previous tev
                pop!(field2d[i].tev)
                #Add a TEV with dummy strength
                place_tev(surf2d[i],field2d[i],dt)

                #Add a LEV with dummy strength
                place_lev(surf2d[i],field2d[i],dt)
                surf2d[i].levflag[1] = 1
                nshed += 1
            else
                surf2d[i].levflag[1] = 0
            end
        end 

In [8]:
if nshed > 0 
            kelvkutta = KelvinKuttaLLTldvm(surf, surf2d, field2d, nshed)

            #Solve for TEV and LEV strengths to satisfy Kelvin condition and Kutta condition at leading edge
            soln = nlsolve(not_in_place(kelvkutta), [-0.01*ones(surf.nspan); 0.01*ones(nshed)])
        end

22 UNSflow.TwoDVort[]
22 UNSflow.TwoDVort[]
23 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-6.80434e-5,0.01,0.02,0.0,0.0)]
24 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.11358e-5,0.01,0.02,0.0,0.0)]
25 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.26066e-5,0.01,0.02,0.0,0.0)]
26 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.34015e-5,0.01,0.02,0.0,0.0)]
27 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.38555e-5,0.01,0.02,0.0,0.0)]
28 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.4131e-5,0.01,0.02,0.0,0.0)]
29 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.42926e-5,0.01,0.02,0.0,0.0)]
30 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.43803e-5,0.01,0.02,0.0,0.0)]
31 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.44067e-5,0.01,0.02,0.0,0.0)]
32 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.43803e-5,0.01,0.02,0.0,0.0)]
33 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.42926e-5,0.01,0.02,0.0,0.0)]
34 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.4131e-5,0.01,0.02,0.0,0.0)]
35 UNSflow.TwoDVort[UN

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01]
 * Zero: [-0.0721315,-0.103178,-0.119087,-0.125714,-0.12777,-0.128893,-0.1295,-0.129876,-0.130091,-0.130212,-0.130246,-0.130212,-0.130091,-0.129876,-0.1295,-0.128893,-0.12777,-0.125714,-0.119087,-0.103178,-0.0721315,0.0105141,0.0184765,0.0209476,0.0222964,0.0230255,0.0234779,0.0237365,0.0238812,0.0239222,0.0238812,0.0237365,0.0234779,0.0230255,0.0222964,0.0209476,0.0184765,0.0105141]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In [9]:
surf2d[1].a0

1-element Array{Float64,1}:
 0.188979

In [10]:
surf2d[21]

UNSflow.TwoDSurf(1.0,1.0,"FlatPlate",0.0,70,35,UNSflow.KinemDef(UNSflow.ConstDef(0.2617993877991494),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0)),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0455303,0.0910607,0.136591,0.182121,0.227652,0.273182,0.318712,0.364243,0.409773  …  2.73182,2.77735,2.82288,2.86841,2.91394,2.95947,3.005,3.05053,3.09606,3.14159],[0.0,0.000518163,0.00207158,0.00465703,0.00826915,0.0129005,0.0185414,0.0251802,0.0328031,0.0413943  …  0.958606,0.967197,0.97482,0.981459,0.9871,0.991731,0.995343,0.997928,0.999482,1.0],UNSflow.KinemPar(0.2617993877991494,0.0,0.0,0.0,1.0,0.0),[-1.0075,-1.007,-1.0055,-1.003,-0.999513,-0.995039,-0.98959,-0.983178,-0.975815,-0.967516  …  -0.081558,-0.0732595,-0.0658963,-0.0594837,-0.0540351,-0.0495616,-0.0460725,-0.0435752,-0.0420747,-0.0415742],[0.0,-0.00013411,-0.000536164,-0.00120533,-0.00214021,-0.003338

In [11]:
field2d[3].lev


1-element Array{UNSflow.TwoDVort,1}:
 UNSflow.TwoDVort(-1.00377,-6.80434e-5,0.0105141,0.02,0.0,0.0)

In [12]:
surf2d[1].lespcrit


1-element Array{Float64,1}:
 0.11

In [13]:
kelv = kelvkutta([-0.01*ones(surf.nspan); 0.01*ones(nshed)])

22 UNSflow.TwoDVort[]
22 UNSflow.TwoDVort[]
23 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-6.80434e-5,0.01,0.02,0.0,0.0)]
24 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.11358e-5,0.01,0.02,0.0,0.0)]
25 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.26066e-5,0.01,0.02,0.0,0.0)]
26 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.34015e-5,0.01,0.02,0.0,0.0)]
27 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.38555e-5,0.01,0.02,0.0,0.0)]
28 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.4131e-5,0.01,0.02,0.0,0.0)]
29 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.42926e-5,0.01,0.02,0.0,0.0)]
30 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.43803e-5,0.01,0.02,0.0,0.0)]
31 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.44067e-5,0.01,0.02,0.0,0.0)]
32 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.43803e-5,0.01,0.02,0.0,0.0)]
33 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.42926e-5,0.01,0.02,0.0,0.0)]
34 UNSflow.TwoDVort[UNSflow.TwoDVort(-1.00377,-7.4131e-5,0.01,0.02,0.0,0.0)]
35 UNSflow.TwoDVort[UN

38-element Array{Float64,1}:
 0.188372 
 0.342997 
 0.461094 
 0.535344 
 0.585324 
 0.618737 
 0.640992 
 0.655604 
 0.664778 
 0.669824 
 0.671435 
 0.669824 
 0.664778 
 ⋮        
 0.100358 
 0.103278 
 0.104885 
 0.105397 
 0.104885 
 0.103278 
 0.100358 
 0.0957068
 0.0886229
 0.0779871
 0.0620781
 0.0384437

In [14]:
    kelv.field[3].lev

LoadError: LoadError: type Array has no field field
while loading In[14], in expression starting on line 1

In [15]:
surf.nspan

21

In [16]:
cntr = surf.nspan + 1

22

In [17]:
cntr += 1 

23